In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torch.distributions import Categorical

# Пример очень простой сети для дискретных действий
class PolicyNetwork(nn.Module):
   def __init__(self, state_dim, action_dim):
       super(PolicyNetwork, self).__init__()
       self.fc1 = nn.Linear(state_dim, 128)
       self.fc2 = nn.Linear(128, action_dim)
       self.activation = nn.ReLU()

   def forward(self, x):
       x = self.fc1(x)
       x = self.activation(x)
       x = self.fc2(x)
       return x # Выход - логиты для softmax

   def act(self, state: np.ndarray):
       """Выбирает действие на основе текущей политики и возвращает действие и его log_prob."""
       state_tensor = torch.from_numpy(state).float().unsqueeze(0)
       logits = self.forward(state_tensor)  # получаем логиты
       dist = Categorical(logits=logits)  # создаем распределение
       action = dist.sample() # Сэмплируем действие
       log_prob = dist.log_prob(action) # Вычисляем log_prob для этого действия
       return action.item(), log_prob

# Инициализация сети и оптимизатора
state_dim = 4  # CartPole state (position, velocity, angle, angular velocity)
action_dim = 2 # CartPole actions (left, right)
policy_net = PolicyNetwork(state_dim, action_dim)

In [ ]:
# Списки для хранения данных одной траектории
rewards = []
log_probs = [] # Здесь будем хранить log(pi(a|s))

state, _ = env.reset() # Сбрасываем среду для нового эпизода
terminated = truncated = False

# Цикл одного эпизода
while not (terminated or truncated):
   action, log_prob = policy_net.act(state) # Агент выбирает действие

   log_probs.append(log_prob) # Сохраняем log_prob

   observation, reward, terminated, truncated, info = env.step(action) # Среда выполняет действие
   rewards.append(reward) # Сохраняем награду
   break

In [ ]:
# Пример вычисления возвратов G_t (discounted returns)
gamma = 0.99
Gt_values = []
current_return = 0
for r in reversed(rewards): # Идем с конца эпизода
    current_return = r + gamma * current_return
    Gt_values.insert(0, current_return) # Вставляем в начало, чтобы сохранить порядок

Gt_values = torch.tensor(Gt_values)
# (Обычно нормализуют Gt_values для стабилизации обучения, но это уже оптимизация)
# Gt_values = (Gt_values - Gt_values.mean()) / (Gt_values.std() + 1e-9)

In [ ]:
# Вычисление функции потерь для Policy Gradient (для градиентного подъема)
# Мы хотим МАКСИМИЗИРОВАТЬ J(theta), что эквивалентно МИНИМИЗАЦИИ -J(theta)
# Поэтому умножаем на -1 и используем .backward()
policy_loss = []
for log_prob, Gt in zip(log_probs, Gt_values):
    policy_loss.append(-log_prob * Gt) # -log_prob потому что оптимизатор минимизирует
policy_loss = torch.cat(policy_loss).sum() # Суммируем по всем шагам траектории

In [ ]:
optimizer.zero_grad() # Обнуляем градиенты перед обратным проходом
policy_loss.backward() # Вычисляем градиенты
optimizer.step() # Обновляем веса сети